In [1]:
!ls ../data

dvdrental      retail_db.sql
dvdrental.zip  retail-sales-index-at-constant-prices-by-industry-quarterly.csv
retail	       retail-sales-index-at-constant-prices-quarterly.csv


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean as _mean, stddev as _stddev, col
from pyspark.sql.types import *

In [3]:
spark = SparkSession \
    .builder \
    .appName("ReadData") \
    .config('spark.driver.extraClassPath',"/home/jovyan/work/jars/postgresql-42.2.24.jar") \
    .config("spark.jars", "/home/jovyan/work/jars/postgresql-42.2.24.jar") \
    .getOrCreate()

21/10/27 06:36:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark.version

'3.2.0'

In [7]:
data = spark.read \
            .format("jdbc") \
            .option("url", "jdbc:postgresql://postgres-data:5432/dvdrental") \
            .option("dbtable", 'customer') \
            .option("user", 'postgres') \
            .option("password", 'postgres') \
            .option("driver", "org.postgresql.Driver") \
            .load()

In [8]:
data.show(2)

+-----------+--------+----------+---------+--------------------+----------+----------+-----------+--------------------+------+
|customer_id|store_id|first_name|last_name|               email|address_id|activebool|create_date|         last_update|active|
+-----------+--------+----------+---------+--------------------+----------+----------+-----------+--------------------+------+
|        524|       1|     Jared|      Ely|jared.ely@sakilac...|       530|      true| 2006-02-14|2013-05-26 14:49:...|     1|
|          1|       1|      Mary|    Smith|mary.smith@sakila...|         5|      true| 2006-02-14|2013-05-26 14:49:...|     1|
+-----------+--------+----------+---------+--------------------+----------+----------+-----------+--------------------+------+
only showing top 2 rows



In [ ]:
df = spark.read \
    .csv('../data/retail-sales-index-at-constant-prices-by-industry-quarterly.csv',header=True)

In [ ]:
type(df)

In [ ]:
df.printSchema()

In [ ]:
df.show(5)

In [ ]:
df = df.withColumnRenamed('level_1','level_one')

In [ ]:
df.show(2)

In [ ]:
df_stat = df.select(
    _mean(col('value')).alias('mean'),
    _stddev(col('value')).alias('std')
    ).collect()

In [ ]:
print(df_stat)

In [ ]:
mean = df_stat[0]['mean']
std = df_stat[0]['std']
print('Mean:',mean)
print('STD:',std)

In [ ]:
df.na.fill(mean,subset=['value']).show()

In [ ]:
df.filter("value = '0' and quarter = '1985-Q1'").count()

In [ ]:
df = df.withColumn('value',regexp_replace('value','na','0'))
df = df.withColumn('value',col('value').astype(IntegerType()))
df = df.withColumn('day',date_add(current_date(),+1))
df.show(5)

In [ ]:
df.write.mode('append').partitionBy('day').format("parquet").save('../data/retail')

In [ ]:
df.drop('day').show(5)

In [ ]:
df.distinct().count()

In [ ]:
df.dropDuplicates().count()